In [1]:
import librosa
import tensorflow as tf
import numpy as np
from ai_edge_litert.interpreter import Interpreter

2025-05-16 10:32:37.951765: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-16 10:32:37.973938: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747405957.997398    4151 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747405958.004275    4151 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747405958.023881    4151 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [7]:
encoder = "/home/dylenthomas/whisperProject/whisper/.model/WhisperEncoder.tflite"
decoder = "/home/dylenthomas/whisperProject/whisper/.model/WhisperDecoder.tflite"
audio = "/home/dylenthomas/whisperProject/whisper/8455-210777-0068.wav"
full_model = "/home/dylenthomas/whisperProject/whisper/.model/whisper-small.en.tflite"

def print_details(details, type):
    print("-"*10 + f" {type} details ".upper() + "-"*10)
    for detail in details:
        print(detail)

In [3]:
test = Interpreter(model_path=full_model)

test_input_details = test.get_input_details()
test_output_details = test.get_output_details()

print_details(test_input_details, "test input")
print_details(test_output_details, "test output")

---------- TEST INPUT DETAILS ----------
{'name': 'serving_default_input_features:0', 'index': 0, 'shape': array([   1,   80, 3000], dtype=int32), 'shape_signature': array([   1,   80, 3000], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
---------- TEST OUTPUT DETAILS ----------
{'name': 'StatefulPartitionedCall:0', 'index': 1838, 'shape': array([  1, 449], dtype=int32), 'shape_signature': array([  1, 449], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}


In [48]:
from utils.WhisperProcessor import offlineWhisperProcessor
import torch

signal, _ = librosa.load(audio, sr=16000, mono=True)

processor = offlineWhisperProcessor("/home/dylenthomas/whisperProject/whisper/utils/preprocessor_config.json",
                                    "/home/dylenthomas/whisperProject/whisper/utils/tokenizer_config.json",
                                    "/home/dylenthomas/whisperProject/whisper/utils/vocab.json")
signal_test = torch.Tensor(signal)

waveform_features = processor.extract_features(signal_test)
print((waveform_features))

tensor([[[-0.1483, -0.0561,  0.0037,  ..., -0.6740, -0.6740, -0.6740],
         [-0.2433,  0.0427, -0.0107,  ..., -0.6740, -0.6740, -0.6740],
         [ 0.0475,  0.0221,  0.0333,  ..., -0.6740, -0.6740, -0.6740],
         ...,
         [-0.6740, -0.6740, -0.6740,  ..., -0.6740, -0.6740, -0.6740],
         [-0.6740, -0.6740, -0.6740,  ..., -0.6740, -0.6740, -0.6740],
         [-0.6740, -0.6740, -0.6740,  ..., -0.6740, -0.6740, -0.6740]]])


In [49]:
test.allocate_tensors()
test.set_tensor(test_input_details[0]["index"], waveform_features)

with tf.device("/CPU:0"):
    test.invoke()

In [50]:
output = test.get_tensor(test_output_details[0]["index"])[0, :]

print(output)
for tok in output:
    print(processor.decode_single(tok))

[50257 50363  3406  1176   318  6751    11   314   531    13 50469 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50

In [11]:
encoder_interpreter = Interpreter(model_path=encoder)
decoder_interpreter = Interpreter(model_path=decoder)

encoder_input_details = encoder_interpreter.get_input_details()
encoder_output_details = encoder_interpreter.get_output_details()
decoder_input_details = decoder_interpreter.get_input_details()
decoder_output_details = decoder_interpreter.get_output_details()



print_details(encoder_input_details, "encoder input")
print_details(encoder_output_details, "encoder output")
print_details(decoder_input_details, "decoder input")
print_details(decoder_output_details, "decoder output")

---------- ENCODER INPUT DETAILS ----------
{'name': 'audio', 'index': 0, 'shape': array([   1,   80, 3000], dtype=int32), 'shape_signature': array([   1,   80, 3000], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
---------- ENCODER OUTPUT DETAILS ----------
{'name': 'k_cache', 'index': 387, 'shape': array([   4,    6,   64, 1500], dtype=int32), 'shape_signature': array([   4,    6,   64, 1500], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
{'name': 'v_cache', 'index': 420, 'shape': array([   4,    6, 1500,   64], dtype=int32), 'shape_signature': array([   4,    6, 1500,   64], dtype=int32), 'dtype': <class 'numpy.float32'

In [12]:
encoder_interpreter.allocate_tensors()

#set the audio tensor for inference
encoder_interpreter.set_tensor(encoder_input_details[0]["index"], waveform_features)

with tf.device("/CPU:0"):
    #run inference
    encoder_interpreter.invoke()

k_cache = encoder_interpreter.get_tensor(encoder_output_details[0]["index"])
v_cache = encoder_interpreter.get_tensor(encoder_output_details[1]["index"])

print("k_cache shape: {}".format(k_cache.shape))
print("v_cache shape: {}".format(v_cache.shape))

k_cache shape: (4, 6, 64, 1500)
v_cache shape: (4, 6, 1500, 64)


In [47]:
decoder_interpreter.allocate_tensors()

#set the start (or last predicted token) token
decoder_interpreter.set_tensor(decoder_input_details[0]["index"], np.array([[50258]], np.int32))
#set the starting index
decoder_interpreter.set_tensor(decoder_input_details[1]["index"], np.array([[0]], np.int32)) # seems to like to start at 1
#set the k and v caches for decoding
decoder_interpreter.set_tensor(decoder_input_details[2]["index"], k_cache)
decoder_interpreter.set_tensor(decoder_input_details[3]["index"], v_cache)
#initialize self attn
decoder_interpreter.set_tensor(decoder_input_details[4]["index"], np.zeros((4, 6, 64, 224), np.float32))
decoder_interpreter.set_tensor(decoder_input_details[5]["index"], np.zeros((4, 6, 224, 64), np.float32))
#run inference
decoder_interpreter.invoke()

logits = decoder_interpreter.get_tensor(decoder_output_details[0]["index"])
prev_k_cache = decoder_interpreter.get_tensor(decoder_output_details[1]["index"])
prev_v_cache = decoder_interpreter.get_tensor(decoder_output_details[2]["index"])

probabilities = tf.nn.softmax(logits, axis=-1)
val = tf.argmax(probabilities, axis=-1).numpy()
val = val.item()
print(processor.decode_single(val))

# I think the pipeline for this model is you feed the token that came out of the decoder aswell as the encoded signal back in to the decoder (without encoding agian) with self attention until it hits the end of transcript token

<|endoftext|>


### Find another tflite encoder decoder file

In [ ]:
not_end = True
i = 0
token = 50258 #<|startoftranscript|>
print("===Start Token===")
print(processor.decode_single(token), token)
print("="*6)

decoder_interpreter.allocate_tensors()

prev_k_cache = np.zeros((4, 6, 64, 224), np.float32)
prev_v_cache = np.zeros((4, 6, 224, 64), np.float32)

while not_end:
    decoder_interpreter.set_tensor(decoder_input_details[0]["index"], np.array([[token]], np.int32))
    decoder_interpreter.set_tensor(decoder_input_details[1]["index"], np.array([[i]], np.int32))
    decoder_interpreter.set_tensor(decoder_input_details[2]["index"], k_cache)
    decoder_interpreter.set_tensor(decoder_input_details[3]["index"], v_cache)
    decoder_interpreter.set_tensor(decoder_input_details[4]["index"], prev_k_cache)
    decoder_interpreter.set_tensor(decoder_input_details[5]["index"], prev_v_cache)

    with tf.device("/CPU:0"):
        #inference
        decoder_interpreter.invoke()

    logits = decoder_interpreter.get_tensor(decoder_output_details[0]["index"])
    prev_k_cache = decoder_interpreter.get_tensor(decoder_output_details[1]["index"])
    prev_v_cache = decoder_interpreter.get_tensor(decoder_output_details[2]["index"])

    #probabilities = tf.nn.softmax(logits, axis=-1)
    #token = tf.argmax(probabilities, axis=-1).numpy().item()
    token = np.argmax(logits[0, -1])
    print(processor.decode_single(token), token)

    if token == 50257:
        not_end = False

    i += 1

===Start Token===
<|startoftranscript|> 50258
k_cache mean/std: 0.00564328 0.2848788
v_cache mean/std: 0.010671045 0.34908122
<|endoftext|> 50257


For now I can deploy using this:

In [39]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

import os
os.environ["TRANSFORMERS_OFFLINE"] = "1"

In [41]:
predicted = model.generate(waveform_features)[0]

for tok in predicted:
    print(processor.decode_single(tok))

ĠYour
Ġpower
Ġis
Ġsufficient
,
ĠI
Ġsaid
.
